# DINO Clustering

In [ ]:
import gc
import os
import sys
from copy import deepcopy
from time import sleep, time

import kornia as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycolmap
from matplotlib.lines import Line2D
from sklearn.metrics import silhouette_score
from IPython.display import clear_output
from scripts import utils, database, features

In [ ]:
device = K.utils.get_cuda_device_if_available(0)
print(f"{device=}")

In [ ]:
DATA_DIR = "../data/image-matching-challenge-2025"
DINO_DIR = "weights/dinov2"
OUTPUT_FILE = "train_predictions.csv"

DB_IMG_EXT = ""
DB_CAMERA_MODEL = "simple-pinhole"

# Configure dataset filtering 
DATASETS_FILTER = [
    # New 2025 datasets
    "amy_gardens",
    "ETs",
    "fbk_vineyard",
    "stairs",
    # Data from IMC 2023 and 2024.
    'imc2024_dioscuri_baalshamin',
    'imc2023_theather_imc2024_church',
    'imc2023_heritage',
    'imc2023_haiper',
    'imc2024_lizard_pond',
    # Crowdsourced PhotoTourism data.
    'pt_stpeters_stpauls',
    'pt_brandenburg_british_buckingham',
    'pt_piazzasanmarco_grandplace',
    'pt_sacrecoeur_trevi_tajmahal',
]

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, "train_labels.csv"))

In [ ]:
# Load the dataset
samples = utils.dataset.load_dataset(DATA_DIR)

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

## Clustering Analysis

In [ ]:
datasets = {}
for dataset in df['dataset'].unique():
    datasets[dataset] = {}
    dataset_rows = df[df['dataset'] == dataset]
    for scene in dataset_rows['scene'].unique():
        datasets[dataset][scene] = []
        scene_rows = dataset_rows[dataset_rows['scene'] == scene]
        image_paths = [os.path.join(DATA_DIR, 'train', dataset, path) for path in scene_rows['image']]
        print(f"Generating descriptors for {dataset}/{scene} -> num_images={len(image_paths)}")
        global_descriptors = features.extraction.extract_cls_descriptor_dino(
            image_paths,
            dino_path=DINO_DIR,
            device=device,
            normalize=True
        )
        datasets[dataset][scene] = global_descriptors

In [ ]:
all_vectors = {dataset_name: [] for dataset_name in datasets.keys()}
all_labels = {dataset_name: [] for dataset_name in datasets.keys()}
all_label_names = {dataset_name: [] for dataset_name in datasets.keys()}

for dataset_name, scenes in datasets.items():
    for cluster_id, (name, vectors) in enumerate(scenes.items()):
        all_vectors[dataset_name].extend([v.cpu().numpy() for v in vectors])
        all_labels[dataset_name].extend([cluster_id] * len(vectors))
        all_label_names[dataset_name].extend([name] * len(vectors))

In [ ]:
for dataset_name, image_features in all_vectors.items():
    # Step 1: t-SNE on features
    features_2d = features.extraction.feature_reducer(
            algorithm="TSNE",
            features=np.vstack(image_features),
            n_components=2,
            random_state=42,
            perplexity=15
        )

    # Step 2: Run HDBSCAN or any clustering on 2D features
    cluster_labels = features.clustering.dino_clusterer(
            algorithm="HDBSCAN",
            features=features_2d,
            scaler=None,
            min_cluster_size=2,
        )
    silhouette_avg = silhouette_score(features_2d, cluster_labels)
    print(f"Dataset: {dataset_name}, Silhouette Score: {silhouette_avg:.4f}")

    # Step 3: Prepare GT label color map
    gt_labels = all_label_names[dataset_name]  # Make sure this is aligned with image_features
    cluster_names = sorted(set(gt_labels))
    gt_palette = sns.color_palette("hls", len(cluster_names))
    gt_color_map = {name: gt_palette[i] for i, name in enumerate(cluster_names)}
    gt_colors = [gt_color_map[label] for label in gt_labels]

    # Step 4: Prepare prediction label color map
    unique_labels = sorted(set(cluster_labels))
    pred_palette = sns.color_palette("hls", len(unique_labels))
    pred_color_map = {label: pred_palette[i] for i, label in enumerate(unique_labels)}
    pred_colors = [pred_color_map[label] for label in cluster_labels]

    # ---- Plotting both subplots ----
    fig, axs = plt.subplots(1, 2, figsize=(18, 8))
    fig.suptitle(f"t-SNE Clustering: {dataset_name}", fontsize=16)

    # --- Left: Prediction ---
    ax = axs[0]
    for label in unique_labels:
        indices = [i for i, l in enumerate(cluster_labels) if l == label]
        subset = features_2d[indices]
        ax.scatter(subset[:, 0], subset[:, 1],
                   c=[pred_color_map[label]],
                   label=f'Cluster {label}' if label != -1 else 'Outlier',
                   s=40, alpha=0.6, edgecolors='k')
    ax.set_title("Predicted Clusters")
    ax.set_xlabel("t-SNE 1")
    ax.set_ylabel("t-SNE 2")
    ax.legend(fontsize='small')

    # --- Right: Ground Truth ---
    ax = axs[1]
    for name in cluster_names:
        indices = [i for i, lbl in enumerate(gt_labels) if lbl == name]
        subset = features_2d[indices]
        ax.scatter(subset[:, 0], subset[:, 1],
                   c=[gt_color_map[name]],
                   label=name,
                   s=40, alpha=0.6, edgecolors='k')
    ax.set_title("Ground Truth Clusters")
    ax.set_xlabel("t-SNE 1")
    ax.set_ylabel("t-SNE 2")

    # GT Legend
    handles = [
        Line2D([0], [0], marker='o', color='w', label=label,
               markerfacecolor=gt_color_map[label], markersize=8, markeredgecolor='k')
        for label in cluster_names
    ]
    ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small', title='GT')

    plt.tight_layout()
    plt.show()

## Do Inference

In [ ]:
# Clear memory to prevent OOM errors
gc.collect()
mapping_result_strs = []  # Store results for each dataset

print(f"Extracting on device {device}")
# Process each dataset
for dataset, predictions in samples.items():
    # Skip datasets not in filter list
    if DATASETS_FILTER and dataset not in DATASETS_FILTER:
        print(f'Skipping "{dataset}"')
        continue

    # Setup paths and image lists
    images_dir = os.path.join(DATA_DIR, "train", dataset)
    images = [os.path.join(images_dir, p.filename) for p in predictions]

    print(f'\nProcessing dataset "{dataset}": {len(images)} images')

    # Map filenames to prediction indices
    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}

    # Setup output directory for features
    feature_dir = os.path.join("featureout", dataset)
    os.makedirs(feature_dir, exist_ok=True)

    # Wrap algos in try-except blocks so we can populate a submission even if one scene crashes.
    try:
        # 1. Image pair selection using DINO features
        t = time()
        cls_descriptor_dino = features.extraction.extract_cls_descriptor_dino(
            images,
            dino_path=DINO_DIR,
            device=device,
            normalize=True
        )
        reduced_features = features.extraction.feature_reducer(
            algorithm="PCA",
            features=cls_descriptor_dino.cpu().numpy(),
            n_components=50,
            scaler=None,
            random_state=42
        )
        cluster_labels = features.clustering.dino_clusterer(
            algorithm="HDBSCAN",
            features=reduced_features,
            scaler=None,
            min_cluster_size=10,
            metric='euclidean',
            cluster_selection_method='eom',
        )
        print(
            f"Clustering. Number of clusters: {np.unique(cluster_labels)}, with {sum(cluster_labels == -1)} outliers. Done in {time() - t:.4f} sec"
        )
        gc.collect()
        images_np = np.array(images)[cluster_labels != -1]
        cluster_labels = cluster_labels[cluster_labels != -1]
        for cluster in np.unique(cluster_labels):
            cluster_images = images_np[cluster_labels == cluster]
            feature_dir_cluster = os.path.join(feature_dir, f"cluster_{cluster}")
            os.makedirs(feature_dir_cluster, exist_ok=True)

            print(f"Processing Cluster {cluster}: {len(cluster_images)} images")

            index_pairs = features.matching.get_image_pairs_shortlist_dino(
                cluster_images.tolist(),
                dino_path=DINO_DIR,
                sim_th=0.3,  # Strict similarity threshold
                min_pairs=20,  # Minimum pairs per image with biggest similarity
                exhaustive_if_less=20,
                device=device,
            )
            print(
                f"Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec"
            )
            gc.collect()

            # 2. Local feature detection with ALIKED
            t = time()
            features.extraction.detect_keypoint_aliked(images, feature_dir_cluster, 4096, device=device)
            gc.collect()
            print(f"Features detected in {time() - t:.4f} sec")

            # 3. Feature matching with LightGlue
            t = time()
            features.matching.match_keypoint_lightglue(
                images, index_pairs, feature_dir=feature_dir_cluster, device=device, verbose=False
            )
            print(f"Features matched in {time() - t:.4f} sec")

            # 4. Create/reset COLMAP database
            database_path = os.path.join(feature_dir_cluster, "colmap.db")
            if os.path.isfile(database_path):
                os.remove(database_path)
            gc.collect()
            sleep(1)
            # Import features and matches into COLMAP format
            database.h5_to_db.import_into_colmap(
                images_dir,
                DB_CAMERA_MODEL,
                img_ext=DB_IMG_EXT,
                feature_dir=feature_dir_cluster,
                database_path=database_path,
            )
            output_path = f"{feature_dir_cluster}/colmap_rec_aliked"

            # 5. Geometric verification with RANSAC
            t = time()
            pycolmap.match_exhaustive(database_path)
            print(f"Ran RANSAC in {time() - t:.4f} sec")

            # 6. SfM reconstruction with COLMAP
            # Configure reconstruction parameters
            mapper_options = pycolmap.IncrementalPipelineOptions()
            mapper_options.min_model_size = 3  # Allow small reconstructions (min 3 images). Colmap by default does not generate a reconstruction if <10 images are registered.
            mapper_options.max_num_models = 25  # Limit number of separate models
            os.makedirs(output_path, exist_ok=True)
            t = time()
            maps = pycolmap.incremental_mapping(
                database_path=database_path,
                image_path=images_dir,
                output_path=output_path,
                options=mapper_options,
            )
            sleep(1)
            print(f"Reconstruction done in  {time() - t:.4f} sec")
            print(maps)

            clear_output(wait=False)

            # 7. Extract poses from reconstruction
            registered = 0
            for map_index, cur_map in maps.items():
                for _, image in cur_map.images.items():
                    prediction_index = filename_to_index[image.name]
                    # predictions[prediction_index].cluster_index = map_index
                    predictions[prediction_index].cluster_index = cluster
                    predictions[prediction_index].rotation = deepcopy(
                        image.cam_from_world.rotation.matrix()
                    )
                    predictions[prediction_index].translation = deepcopy(
                        image.cam_from_world.translation
                    )
                    registered += 1
            mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images with {len(np.unique(cluster_labels))} clusters'
            mapping_result_strs.append(mapping_result_str)
            print(mapping_result_str)
            print(f"# clusters predicted by colmap: {len(maps)}")
            gc.collect()

    except Exception as e:
        print(e)
        # raise e
        mapping_result_str = f'Dataset "{dataset}" -> Failed!'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)

# Print summary of results
print("\nResults")
for s in mapping_result_strs:
    print(s)

In [ ]:
# Create a submission file.
utils.submission.create_submission_file(samples, OUTPUT_FILE)

!head {OUTPUT_FILE}

In [ ]:
# Compute results if running on the training set.
# Don't do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

t = time()
final_score, dataset_scores = utils.metric.score(
    gt_csv=os.path.join(DATA_DIR, "train_labels.csv"),
    user_csv=OUTPUT_FILE,
    thresholds_csv=os.path.join(DATA_DIR, "train_thresholds.csv"),
    mask_csv=None,
    inl_cf=0,
    strict_cf=-1,
    verbose=True,
)
print(f"Computed metric in: {time() - t:.02f} sec.")